In [1]:
import os
import openai
openai.api_key = "EMPTY"
openai.api_base = "http://172.29.40.143:8000/v1"

In [2]:
contents = ["This is an Apple iPhone." , "Apple is a fruit" , "iPhone is expensive." , "Mango is a fruit"]

In [4]:
received_embeddings = []
for content in contents:
    response = openai.Embedding.create(
        input = content,
        model = "bge-large-en-v1.5",
    )
    received_embeddings.append(response["data"][0]["embedding"])


In [5]:
print(len(received_embeddings))
print(len(received_embeddings[0]))

4
1024


In [40]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
# review how the embeddings look like
embedding1 = np.array(received_embeddings[0])
print(embedding1.shape)
embedding1 = embedding1.reshape(1, -1)
print(embedding1.shape)
print(min(embedding1[0]), max(embedding1[0]))

(4096,)
(1, 4096)
-0.24069924652576447 0.11850529909133911


In [43]:
#contents = ["This is an Apple iPhone." , "Apple is a fruit." , "iPhone is expensive." , "Mango is a fruit"]

In [45]:
# calculate the cosine similarity between the embeddings
embedding1 = np.array(received_embeddings[1]).reshape(1, -1)
embedding2 = np.array(received_embeddings[3]).reshape(1, -1)
similarity  = cosine_similarity(embedding1, embedding2)
print(similarity)

[[0.69816772]]


Compare the embeddings generated by a model running locally with the same model served via FastChat

In [58]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

C:\Users\rochakchadha\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
modules.json: 100%|██████████| 349/349 [00:00<00:00, 348kB/s]
C:\Users\rochakchadha\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rochakchadha\.cache\huggingface\hub\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you ei

In [59]:
local_embeddings = []
for content in contents:
    local_embeddings.append(model.encode(content))

In [61]:
print(len(local_embeddings))
print(len(local_embeddings[0]))

4
1024


In [64]:
# compare received embeddings with local embeddings
for i in range(len(received_embeddings)):
    embedding1 = np.array(received_embeddings[i]).reshape(1, -1)
    embedding2 = np.array(local_embeddings[i]).reshape(1, -1)
    similarity  = cosine_similarity(embedding1, embedding2)
    print(f"cosine similarity: {similarity} for index {i+1}")
    L1 = np.linalg.norm(embedding1 - embedding2, ord=1)
    print(f"L1 norm: {L1} for index {i+1}")
    frobenius = np.linalg.norm(embedding1 - embedding2, ord='fro')
    print(f"Frobenius norm: {frobenius} for index {i+1}")

cosine similarity: [[0.99609623]] for index 1
L1 norm: 0.008976144948974252 for index 1
Frobenius norm: 0.08836028685840333 for index 1
cosine similarity: [[0.98514147]] for index 2
L1 norm: 0.020426571369171143 for index 2
Frobenius norm: 0.17238635083300588 for index 2
cosine similarity: [[0.99442588]] for index 3
L1 norm: 0.010373497381806374 for index 3
Frobenius norm: 0.10558527449177144 for index 3
cosine similarity: [[0.98288568]] for index 4
L1 norm: 0.023268982768058777 for index 4
Frobenius norm: 0.18500981902763577 for index 4


Now run Llama 2 and embeddings server simultaneously on the ASE Pro 2